In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

In [1]:


# Paths to dataset
train_path = "D:/D-Documents/Self-Improvement/Python/Computer Vision/Computer Vision Masterclass/Datasets/cat_dog_2/training_set"
test_path = "D:/D-Documents/Self-Improvement/Python/Computer Vision/Computer Vision Masterclass/Datasets/cat_dog_2test_set"


In [ ]:
img = cv2.imread(train_path + "/cat/cat.1.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
plt.imshow(img)

In [ ]:

# Data Augmentation (similar to ImageDataGenerator)
train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:

# Load datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=test_path, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:

# Display a batch of augmented images
data_iter = iter(train_loader)
images, labels = next(data_iter)

fig, axs = plt.subplots(4, 4, figsize=(10, 10))
for i, ax in enumerate(axs.flat):
    img = images[i].permute(1, 2, 0) * 0.5 + 0.5  # Unnormalize
    ax.imshow(img)
    ax.axis('off')
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ModuleA(nn.Module):
    def __init__(self, in_channels, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
        super(ModuleA, self).__init__()
        self.conv_1x1 = nn.Conv2d(in_channels, filters_1x1, kernel_size=1)
        
        self.conv_3x3_reduce = nn.Conv2d(in_channels, filters_3x3_reduce, kernel_size=1)
        self.conv_3x3 = nn.Conv2d(filters_3x3_reduce, filters_3x3, kernel_size=3, padding=1)
        
        self.conv_5x5_reduce = nn.Conv2d(in_channels, filters_5x5_reduce, kernel_size=1)
        self.conv_5x5_1 = nn.Conv2d(filters_5x5_reduce, filters_5x5, kernel_size=3, padding=1)
        self.conv_5x5_2 = nn.Conv2d(filters_5x5, filters_5x5, kernel_size=3, padding=1)
        
        self.pool_proj = nn.Conv2d(in_channels, filters_pool_proj, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    
    def forward(self, x):
        branch1 = F.relu(self.conv_1x1(x))
        
        branch2 = F.relu(self.conv_3x3_reduce(x))
        branch2 = F.relu(self.conv_3x3(branch2))
        
        branch3 = F.relu(self.conv_5x5_reduce(x))
        branch3 = F.relu(self.conv_5x5_1(branch3))
        branch3 = F.relu(self.conv_5x5_2(branch3))
        
        branch4 = self.pool(x)
        branch4 = F.relu(self.pool_proj(branch4))
        
        return torch.cat([branch1, branch2, branch3, branch4], dim=1)


class ModuleB(nn.Module):
    def __init__(self, in_channels, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
        super(ModuleB, self).__init__()
        self.conv_1x1 = nn.Conv2d(in_channels, filters_1x1, kernel_size=1)
        
        self.conv_3x3_reduce = nn.Conv2d(in_channels, filters_3x3_reduce, kernel_size=1)
        self.conv_3x3_1 = nn.Conv2d(filters_3x3_reduce, filters_3x3, kernel_size=(1, 3), padding=(0, 1))
        self.conv_3x3_2 = nn.Conv2d(filters_3x3, filters_3x3, kernel_size=(3, 1), padding=(1, 0))
        
        self.conv_5x5_reduce = nn.Conv2d(in_channels, filters_5x5_reduce, kernel_size=1)
        self.conv_5x5_1 = nn.Conv2d(filters_5x5_reduce, filters_5x5, kernel_size=(1, 3), padding=(0, 1))
        self.conv_5x5_2 = nn.Conv2d(filters_5x5, filters_5x5, kernel_size=(3, 1), padding=(1, 0))
        self.conv_5x5_3 = nn.Conv2d(filters_5x5, filters_5x5, kernel_size=(1, 3), padding=(0, 1))
        self.conv_5x5_4 = nn.Conv2d(filters_5x5, filters_5x5, kernel_size=(3, 1), padding=(1, 0))
        
        self.pool_proj = nn.Conv2d(in_channels, filters_pool_proj, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    
    def forward(self, x):
        branch1 = F.relu(self.conv_1x1(x))
        
        branch2 = F.relu(self.conv_3x3_reduce(x))
        branch2 = F.relu(self.conv_3x3_1(branch2))
        branch2 = F.relu(self.conv_3x3_2(branch2))
        
        branch3 = F.relu(self.conv_5x5_reduce(x))
        branch3 = F.relu(self.conv_5x5_1(branch3))
        branch3 = F.relu(self.conv_5x5_2(branch3))
        branch3 = F.relu(self.conv_5x5_3(branch3))
        branch3 = F.relu(self.conv_5x5_4(branch3))
        
        branch4 = self.pool(x)
        branch4 = F.relu(self.pool_proj(branch4))
        
        return torch.cat([branch1, branch2, branch3, branch4], dim=1)


class ModuleC(nn.Module):
    def __init__(self, in_channels, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
        super(ModuleC, self).__init__()
        self.conv_1x1 = nn.Conv2d(in_channels, filters_1x1, kernel_size=1)
        
        self.conv_3x3_reduce = nn.Conv2d(in_channels, filters_3x3_reduce, kernel_size=1)
        self.conv_3x3_1 = nn.Conv2d(filters_3x3_reduce, filters_3x3, kernel_size=(1, 3), padding=(0, 1))
        self.conv_3x3_2 = nn.Conv2d(filters_3x3, filters_3x3, kernel_size=(3, 1), padding=(1, 0))
        
        self.conv_5x5_reduce = nn.Conv2d(in_channels, filters_5x5_reduce, kernel_size=1)
        self.conv_5x5 = nn.Conv2d(filters_5x5_reduce, filters_5x5, kernel_size=3, padding=1)
        self.conv_5x5_1 = nn.Conv2d(filters_5x5, filters_5x5, kernel_size=(1, 3), padding=(0, 1))
        self.conv_5x5_2 = nn.Conv2d(filters_5x5, filters_5x5, kernel_size=(3, 1), padding=(1, 0))
        
        self.pool_proj = nn.Conv2d(in_channels, filters_pool_proj, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    
    def forward(self, x):
        branch1 = F.relu(self.conv_1x1(x))
        
        branch2 = F.relu(self.conv_3x3_reduce(x))
        branch2 = F.relu(self.conv_3x3_1(branch2))
        branch2 = F.relu(self.conv_3x3_2(branch2))
        
        branch3 = F.relu(self.conv_5x5_reduce(x))
        branch3 = F.relu(self.conv_5x5(branch3))
        branch3 = F.relu(self.conv_5x5_1(branch3))
        branch3 = F.relu(self.conv_5x5_2(branch3))
        
        branch4 = self.pool(x)
        branch4 = F.relu(self.pool_proj(branch4))
        
        return torch.cat([branch1, branch2, branch3, branch4], dim=1)


class InceptionNetwork(nn.Module):
    def __init__(self):
        super(InceptionNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.module_a1 = ModuleA(192, 16, 64, 128, 16, 32, 32)
        self.module_a2 = ModuleA(256, 64, 64, 192, 32, 96, 64)
        self.module_b1 = ModuleB(448, 16, 64, 128, 16, 32, 32)
        self.module_b2 = ModuleB(256, 64, 64, 192, 32, 96, 64)
        self.module_c1 = ModuleC(192, 16, 64, 128, 16, 32, 32)
        self.module_c2 = ModuleC(256, 64, 64, 192, 32, 96, 64)
        
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(448, 2)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu


In [ ]:

# Initialize the model, loss, and optimizer
model = InceptionNetwork(num_classes=len(train_dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")



In [ ]:

# Test loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")